# 🦌 Text Sizer 📏

This widget is used to send text strings to the browser for sizing. The resulting sizes
are be included in the ElkLabel objects so ELKjs can accuratly place labels in the
diagram.

In [ ]:
import asyncio

import ipywidgets

import ipyelk

In [ ]:
sizer = ipyelk.diagram.ElkTextSizer(max_size=1)
button = ipywidgets.Button(description="Re-Calculate Size")
txt = ipywidgets.Text("hello world", description="Input Text")
output = ipywidgets.Label()
font_size = ipywidgets.IntSlider(description="Font Size", min=10, max=72)
font_family = ipywidgets.Select(
    description="Font Family", options=["sans-serif", "monospace"], rows=1
)


def _on_resize_text(change):
    sizer.style = {
        " text": {
            "font-size": f"{font_size.value}px !important",
            "font-family": f"{font_family.value} !important",
        }
    }
    asyncio.create_task(call_measurement())


[it.observe(_on_resize_text, "value") for it in [font_size, font_family]]


async def call_measurement(*args):
    size = await sizer.measure(txt.value)
    output.value = f"width:{size.width} height:{size.height}"


txt.observe(lambda *args: asyncio.create_task(call_measurement()), "value")
button.on_click(lambda *args: asyncio.create_task(call_measurement()))
ipywidgets.VBox(
    [ipywidgets.HBox([txt, font_size, font_family]), ipywidgets.HBox([button, output])]
)

In [ ]:
sizer.measure.cache_info()

### Responses are cached so that multiple requests for the same label will only be send to the DOM once

In [ ]:
sizer.measure.cache_info()

### Bulk measurements

Can send several requests

In [ ]:
sizer.style

In [ ]:
num = 10
bulk_sizer = ipyelk.diagram.ElkTextSizer(
    max_size=num,
    style={
        " .sprotty .elklabel.larger-font": {"font-size": "20px"},
    },
)

In [ ]:
async def send_bulk_text(num):
    texts = tuple(str(i) for i in range(num))
    await bulk_sizer.measure(texts)
    display(bulk_sizer.measure.cache_info())


asyncio.create_task(send_bulk_text(num))

after running expect to see 10 additional requests captured in the `bulk_sizer.futures`

In [ ]:
bulk_sizer.raw_css

In [ ]:
bulk_sizer.measure.cache_info()

Size labels with css classes applied

In [ ]:
def label(i):
    return ipyelk.diagram.elk_model.ElkLabel(
        id=str(i), text=str(i), properties={"cssClasses": "larger-font"}
    )


async def send_bulk_label(num):
    texts = tuple(label(i) for i in range(num))
    await bulk_sizer.measure(texts)
    display(bulk_sizer.measure.cache_info())


asyncio.create_task(send_bulk_label(num))

## The measure cache can be cleared with

In [ ]:
bulk_sizer.measure.cache_clear()
bulk_sizer.measure.cache_info()

## 🦌 Learn More 📖

See the [other examples](./_index.ipynb).